# 使用数学优化进行共享单车再平衡

共享单车系统已经成为全球城市居民和游客日常出行的有效方式。

NYC的Citi-Bike作为最大的共享单车网络,在2022年7月拥有1,588个活跃站点和25,575辆活跃单车。

2022年7月完成了超过300万次骑行,覆盖NYC/Hoboken/Jersey City区域,约有15万名活跃年度会员。

在高峰时段,许多单车站对单车的需求很高,这意味着这些站点的单车流出量大于流入量。

同时,也有一些站点对车位的需求很高(骑行者将单车归还到这些站点),这意味着这些站点的单车流入量大于流出量。

在高需求站点缺乏可用单车或车位会导致共享单车网络的重大失衡,并造成客户不满和收入损失。

为解决这个问题,需要在站点之间重新调配单车以平衡供需。

## 问题陈述与解决方案

使用2022年7月NYC和Jersey区域的Citi-bike历史数据,我们想知道:
- 8月第一周每个站点每小时的单车需求是多少?
- 了解需求后,如何最小化销售损失?

销售损失是由于客户需要单车时缺乏单车造成的。因此,应该将单车从流入量较大的站点转移到流出量较大的站点。

所以,首先要确定每小时需要向各站点增加或减少的单车数量。然后,安排站点之间的实际单车转运。

在本示例中,我们将专注于第一部分,最后讨论如何解决第二部分。
我们将使用机器学习(ML)和数学优化(MO)的组合来解决这个问题。

**解决方案方法**
解决方案包含两个步骤:
- **步骤1**: 我们使用2022年7月NYC和Jersey区域的Citi-bike历史数据,使用ML模型预测8月第一周每个站点每小时的单车流入和流出量。这在[predict_bike_flow](predict_bike_flow.ipynb)笔记本中完成。
- **步骤2**: 我们使用MO模型来决定每个小时向每个站点增加或减少多少单车,以最小化总销售损失。

为确保每个人都可以使用Gurobi受限许可运行笔记本,我们减少了数据规模。为此,我们在早高峰时段(上午7点到9点)只关注前50个站点。

使用PageRank算法选择了这些顶级站点。

# 安装所需包

In [ ]:
%pip install gurobipy
%pip install pandas

# 导入包

In [ ]:
import datetime
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# 优化问题

## 问题定义

我们希望最小化总销售损失。每个站点在每个小时的销售损失可以定义为总单车需求量(从该站点出发的单车数量)与总供应量之间的差值。

总供应量包括结束行程到达该站点的单车数量,加上站点现有的所有单车(即库存),再加上该小时通过调配增加或减少的单车数量。

**假设:**
- 第一个小时开始时(在本例中为7点)的库存为零。
- 在任何给定小时,我们都可以使用有限数量的单车分配到各站点,以期望在不需要站点之间转移单车的情况下减少失衡(因为这是早高峰时段的分析)。

## 加载所需数据

In [ ]:
stations = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/bike_share/top_stations.csv', index_col='station')
# run locally
# stations = pd.read_csv('top_stations.csv', index_col='station')
stations.head()

In [ ]:
stations_flow = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/bike_share/stations_flow.csv')
# run locally
# stations_flow = pd.read_csv('stations_flow.csv')
stations_flow['datetime'] = pd.to_datetime(stations_flow['datetime'])
stations_flow.head(25)

`stations_flow`数据包含2022年8月前5天所有时段的预测。我们的分析关注早高峰时段,即上午7点到9点。此外,我们可以每天运行MO模型。目前,我们仅关注第一天,但最后我们会展示完整模型并说明如何每天运行。

In [ ]:
# 当创建新列时Pandas会给出一些SettingWithCopyWarning警告
# 这些是误报。所以我们忽略它们。
pd.options.mode.chained_assignment = None
morning_flow = stations_flow[stations_flow['datetime'].dt.hour.between(7, 9)]
morning_flow['date'] = morning_flow['datetime'].dt.date
morning_flow['time'] = morning_flow['datetime'].dt.hour
# 现在,让我们为第一天运行MO模型: 08/01/2022
flow_df = morning_flow.loc[morning_flow['date'] == datetime.date(2022, 8, 1)]
flow_df.set_index(['station', 'time'], inplace=True)
flow_df.head(10)

## 问题建模

让我们定义MO模型的符号。我们希望对每个小时和每个站点运行模型。因此,让我们定义以下两个集合:

**集合**
- $I\quad$: 站点集合
- $T\quad$: 小时集合 

我们还从`stations`和`flow_df`数据框中获得以下信息:

**参数**
- $e_{i,t}\quad$: 在t小时结束行程到达站点i的单车数量(即供应)
- $s_{i,t}\quad$: 在t小时从站点i开始行程的单车数量(即需求)
- $c_{i}\quad$: 站点i的容量

我们知道在高峰时段和拥堵时段,在站点之间转移单车并不容易。为了减少高需求站点因缺乏单车而造成的损失,我们假设每小时开始时都有少量的备用单车可以分配给各站点。我们用$N$表示:
- $N\quad$: 在给定小时内可以分配给站点的备用单车数量

In [ ]:
num_bikes = 25  # N: 在给定小时内可以分配给站点的备用单车数量
station_names = list(stations.index)  # 集合 I
time_rng = morning_flow['time'].drop_duplicates().values  # 集合 T

In [ ]:
station_time = flow_df.index  # (i,t)索引对
start_forecast = flow_df.start_forecast  # s
end_forecast = flow_df.end_forecast  # e
capacity = stations.capacity  # c

## 决策变量

首先,我们想要确定每个站点在每个小时内需要增加或减少的单车数量。我们用$y_{i,t}$表示增加的单车数量,用$z_{i,t}$表示减少的单车数量:
- $y_{i,t}\quad$: 在t小时向站点i增加的单车数量
- $z_{i,t}\quad$: 在t小时从站点i减少的单车数量

模型的目标是减少每个站点每小时的销售损失。这个值也取决于决策变量$y_{i,t}$和$z_{i,t}$的值,因此它本身也是一个决策变量。我们用$l_{i,t}$表示:
- $l_{i,t}\quad$: 站点i在t小时的销售损失

**你能想到其他变量吗?**

有了这些初始决策变量,我们就可以开始构建MO模型了:

In [ ]:
mdl = gp.Model('bike_rebalancing')
# 变量

y = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='y')
# 替代方案:
# y = mdl.addVars(station_names, time_rng, lb=0, vtype=GRB.CONTINUOUS, name='y')
z = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='z')
l = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='l')

## 约束条件

首先,我们为每小时可以增加或减少的单车数量设置上下限。

在每小时内,我们有$s_{i,t}$辆单车从站点出发,$e_{i,t}$辆单车到达站点。如果$s_{i,t} \ge e_{i,t}$,需求超过供应。在这种情况下,我们可以选择向该站点增加一些单车以减少损失(也可以选择不增加)。需要记住的是,我们不能增加超过站点容量的单车。

另一方面,如果$s_{i,t} \le e_{i,t}$,我们可以选择从该站点减少一些多余的单车(也可以选择不减少)。然而,如果$e_{i,t} \ge s_{i,t} + c_i$,那么到达站点的单车数量将超过站点容量。在这种情况下,我们必须减少一些单车以避免溢出。

上述描述帮助我们定义了$y_{i,t}$和$z_{i,t}$的界限。
对于增加单车,界限是:

\begin{align}
&??? &\quad \forall i \in I, t \in T\\
\end{align}

对于减少单车:

\begin{align}
&??? &\quad \forall i \in I, t \in T\\
\end{align}


In [ ]:
# Y的界限
... # ???

# z的界限
... # ???

接下来,我们将初始库存(即$t_0$小时的库存)设置为0。

\begin{align}
&??? & \\
\end{align}

In [ ]:
t0 = 7
# 设置初始库存
... # ???

如何定义一个小时开始时站点的库存?库存量是否有上限?


\begin{align}
&??? &\\
\end{align}



接下来,我们需要定义销售损失的计算方法。销售损失是站点需求与供应之间的差值。
- 需求是所有离开站点的单车数量。那么,需求是什么?
- 供应是所有到达站点的单车数量。那么,供应是什么?

当然,如果供应大于需求,则没有损失。因此,我们需要确保损失只考虑非负值。这可以通过以下方式实现:

\begin{align}
&??? &\quad \forall i \in I, t \in T \\
\end{align}

In [ ]:
# 损失定义
... # ???

我们假设每小时开始时有少量备用单车可以分配给各站点。这个限制是对分配给站点的单车总数的限制。

你认为我们需要做什么来添加这个约束?


\begin{align}
&??? \\
\end{align}

In [ ]:
# 限制增加的单车数量
... # ???

## 目标

目标是最小化总销售损失。

$$\min \sum_{i,t} l_{i,t}$$

In [ ]:
mdl.setObjective(l.sum(), GRB.MINIMIZE)

我们现在可以告诉Gurobi模型已经完成,可以解决问题了。

In [ ]:
mdl.optimize()

## 后处理

In [ ]:
df = pd.DataFrame()
if mdl.status == GRB.Status.OPTIMAL:
    df = flow_df.copy()
    df = df.merge(stations[['capacity']], left_on='station', right_index=True)
    df[['bikes_added', 'bikes_removed', 'loss_sale', 'beginning_inventory']] = 0
    for k, v in y.items():
        df.loc[k, 'bikes_added'] = v.x
        df.loc[k, 'bikes_removed'] = z[k].x
        df.loc[k, 'beginning_inventory'] = q[k].x
        df.loc[k, 'loss_sale'] = l[k].x
    df.reset_index(inplace=True)
    print(f'Total Loss : {mdl.objVal}')
    total_bikes_added = df.groupby('time')['bikes_added'].sum()
    print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
else:
    print('Could not find a feasible solution!')
df.head(10)

# 模型增强

查看`output_df`,你可能会注意到每小时都有单车转移(增加或减少)的站点。更糟糕的是,有些站点在一个小时内减少单车,但在下一个小时内增加单车。我们知道这些增加/减少消耗时间和金钱。我们可以做些什么来避免这种情况?

一种方法是引入固定成本,用于转移单车的卡车(或自行车拖车),然后将此项添加到目标函数中。通过与转移相关的成本,模型被激励使用更少的转移次数。

首先,我们应该计算转移发生的次数。每当向站点增加单车或从站点减少单车时,就发生了一次转移。因此,我们需要一种方法将单车的增加和减少与转移联系起来。实现这一目标的一种方法是引入两个新的二元变量:

- $x_{i,t}\quad$: 如果在t小时向站点i增加单车,则为1;否则为0
- $w_{i,t}\quad$: 如果在t小时从站点i减少单车,则为1;否则为0

接下来,我们需要建立$y_{i,t}$与$x_{i,t}$之间的关系,以及$z_{i,t}$与$w_{i,t}$之间的关系。基本上,我们想说:
如果$y_{i,t} \ge 0$,则$x_{i,t} = 1$;如果$z_{i,t} \ge 0$,则$w_{i,t} = 1$。

我们引入以下两个约束条件:

\begin{align}
&y_{i,t} \le M x_{i,t} &\quad \forall i \in I, t \in T \tag{11}\\
&z_{i,t} \le M w_{i,t} &\quad \forall i \in I, t \in T \tag{12}\\
\end{align}

其中$M$是一个大数。

约束条件11确保如果$y_{i,t}\ge 0$,则$x_{i,t} = 1$。但单独来看,这个约束条件不能使$x_{i,t} =0$如果$y_{i,t} \le 0$。约束条件12也是如此。它确保$z_{i,t} \ge 0$使$w_{i,t} = 1$。然而,它不能强制$w_{i,t} = 0$如果$z_{i,t} \ge 0$。

这可以通过目标函数实现。

总转移次数等于$x_{i,t}$和$w_{i,t}$的总和,我们的目标是最小化转移次数。因此,我们将这些项添加到目标函数中。换句话说,我们的新目标函数是:

$$\min \sum_{i,t} (l_{i,t} + x_{i,t} + w_{i,t})$$

由于最小化总转移次数是目标,目标函数试图使$x_{i,t}$和$w_{i,t}$尽可能小(在这种情况下为零)。结合约束条件11和12,这意味着在$x_{i,t}$和$w_{i,t}$可以取0或1的情况下,目标函数迫使它们取值为0。此外,由于任何额外的转移会导致$x_{i,t}$或$w_{i,t}$为1,模型被激励减少转移次数以最小化目标函数。

当然,你可以通过为目标函数中的每个项设置系数(将其视为成本)使模型更加通用。

In [ ]:
x = mdl.addVars(station_time, vtype=GRB.BINARY, name='x')  # 如果 y_{i,t} >= 0 则为1
w = mdl.addVars(station_time, vtype=GRB.BINARY, name='w')  # 如果 z_{i,t} >= 0 则为1

big_m = 1000  # 大数
# y和x之间的关系
mdl.addConstrs((y[i, t] <= big_m * x[i, t] for i, t in station_time), 'rel_y_x')
# z和w之间的关系
mdl.addConstrs((z[i, t] <= big_m * w[i, t] for i, t in station_time), 'rel_z_w')

# 新的目标函数
obj = l.sum() + (x.sum() + w.sum())
mdl.setObjective(obj, GRB.MINIMIZE)
mdl.optimize()

## 后处理

In [ ]:
df = pd.DataFrame()
if mdl.status == GRB.Status.OPTIMAL:
    df = flow_df.copy()
    df = df.merge(stations[['capacity']], left_on='station', right_index=True)
    df[['bikes_added', 'bikes_removed', 'loss_sale', 'beginning_inventory']] = 0
    for k, v in y.items():
        df.loc[k, 'bikes_added'] = v.x
        df.loc[k, 'bikes_removed'] = z[k].x
        df.loc[k, 'beginning_inventory'] = q[k].x
        df.loc[k, 'loss_sale'] = l[k].x
    df.reset_index(inplace=True)
    print(f'Total Loss : {mdl.objVal}')
    total_bikes_added = df.groupby('time')['bikes_added'].sum()
    print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
else:
    print('Could not find a feasible solution!')
df.head(10)

# 额外

## 场景分析

在许多MO问题中,场景分析或假设分析是一个重要的需求。
一般来说,在假设分析中,我们感兴趣的是在各种场景下解决方案如何变化。
想想我们的案例。
- 如果备用单车数量增加或减少10%,解决方案会如何变化?
- 在增强模型中,如果销售损失的成本发生变化会怎样?增加或减少单车的成本又会怎样?
- 如果在最繁忙的站点附近增加一个新站点会怎样?
- 如果我们希望确保每个站点在每小时开始时至少有2辆可用单车会怎样?

我们的模型仍然是一个简单的模型。但你可以想象场景分析可以提供的价值。它可以通过创建和比较不同的场景并评估其结果来回答许多问题,从而评估其对业务目标的影响。要了解更多,你可以查看Gurobi的这个[多场景示例](https://www.gurobi.com/documentation/9.5/examples/multiscenario_py.html)。

## 现实中如何解决这个问题?

在知道每个站点需要多少单车后,需要将单车从一个站点实际移动到另一个站点。

在高峰时段,交通已经很拥堵。因此,使用自行车拖车(通常可以容纳5辆自行车)来移动单车。

在较轻的时段(主要是晚上),使用卡车转移单车。

无论哪种情况,都需要从单车流入量较大的站点移除一些单车,并将其转移到单车流出量较大的站点以平衡它们。
这个问题,即卡车需要从一个站点到另一个站点并接送单车,本身就是另一个数学优化问题。

在这个问题中,我们需要确保所有需要接送的站点在特定时间窗口内被访问,目标可以是使用最少数量的卡车或最小的运输成本(例如,燃料成本加上使用卡车的成本)。这个问题是著名的带接送的车辆路径问题(VRP)的变体。
要了解更多,请查看[这个网络研讨会](https://www.gurobi.com/resource/how-to-synchronize-complex-routing-operations-synched-vrps-with-gurobi/)